# Spark Dataframe Agent

This notebook shows how to use agents to interact with a Spark dataframe and Spark Connect. It is mostly optimized for question answering.

**NOTE: this agent calls the Python agent under the hood, which executes LLM generated Python code - this can be bad if the LLM generated Python code is harmful. Use cautiously.**

In [5]:
from langchain.agents import create_spark_dataframe_agent
import os

os.environ["OPENAI_API_KEY"] = "...input_your_openai_api_key..."

In [11]:
from langchain.llms import OpenAI
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
csv_file_path = "titanic.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
df.show()

ConnectionRefusedError: [Errno 61] Connection refused

In [ ]:
agent = create_spark_dataframe_agent(llm=OpenAI(temperature=0), df=df, verbose=True)


In [17]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: I need to find out how many rows are in the dataframe
Action: python_repl_ast
Action Input: df.count()
Observation: 891
Thought: I now know the final answer
Final Answer: There are 891 rows in the dataframe.

> Finished chain.


'There are 891 rows in the dataframe.'

In [12]:
agent.run("how many people have more than 3 siblings")



> Entering new AgentExecutor chain...
Thought: I need to find out how many people have more than 3 siblings
Action: python_repl_ast
Action Input: df.filter(df.SibSp > 3).count()
Observation: 30
Thought: I now know the final answer
Final Answer: 30 people have more than 3 siblings.

> Finished chain.


'30 people have more than 3 siblings.'

In [13]:
agent.run("whats the square root of the average age?")



> Entering new AgentExecutor chain...
Thought: I need to get the average age first
Action: python_repl_ast
Action Input: df.agg({"Age": "mean"}).collect()[0][0]
Observation: 29.69911764705882
Thought: I now have the average age, I need to get the square root
Action: python_repl_ast
Action Input: math.sqrt(29.69911764705882)
Observation: name 'math' is not defined
Thought: I need to import math first
Action: python_repl_ast
Action Input: import math
Observation: 
Thought: I now have the math library imported, I can get the square root
Action: python_repl_ast
Action Input: math.sqrt(29.69911764705882)
Observation: 5.449689683556195
Thought: I now know the final answer
Final Answer: 5.449689683556195

> Finished chain.


'5.449689683556195'

# Spark Connect Example

In [ ]:
# in apache-spark root directory. (tested here with "spark-3.4.0-bin-hadoop3 and later")
# To launch Spark with support for Spark Connect sessions, run the start-connect-server.sh script.
!./sbin/start-connect-server.sh --packages org.apache.spark:spark-connect_2.12:3.4.0

In [ ]:
from pyspark.sql import SparkSession

# Now that the Spark server is running, we can connect to it remotely using Spark Connect. We do this by 
# creating a remote Spark session on the client where our application runs. Before we can do that, we need 
# to make sure to stop the existing regular Spark session because it cannot coexist with the remote 
# Spark Connect session we are about to create.
SparkSession.builder.master("local[*]").getOrCreate().stop()

In [4]:
# The command we used above to launch the server configured Spark to run as localhost:15002. 
# So now we can create a remote Spark session on the client using the following command.
spark = SparkSession.builder.remote("sc://localhost:15002").getOrCreate()

In [ ]:
csv_file_path = "titanic.csv"
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)
df.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [ ]:
from langchain.agents import create_spark_connect_dataframe_agent
from langchain.llms import OpenAI
import os

os.environ["OPENAI_API_KEY"] = "...input your OpenAI API key here..."

agent = create_spark_connect_dataframe_agent(llm=OpenAI(temperature=0), df=df, verbose=True)

In [ ]:
agent.run("""
who bought the most expensive ticket?
You can find all supported function types in https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/dataframe.html
""")



> Entering new AgentExecutor chain...

Thought: I need to find the ticket with the highest fare
Action: python_repl_ast
Action Input: df.agg({"Fare": "max"}).collect()
Observation: [Row(max(Fare)=512.3292)]
Thought: I need to find the ticket with the highest fare
Action: python_repl_ast
Action Input: df.filter(df.Fare == 512.3292).collect()
Observation: [Row(PassengerId=259, Survived=1, Pclass=1, Name='Ward, Miss. Anna', Sex='female', Age=35.0, SibSp=0, Parch=0, Ticket='PC 17755', Fare=512.3292, Cabin=None, Embarked='C'), Row(PassengerId=680, Survived=1, Pclass=1, Name='Cardeza, Mr. Thomas Drake Martinez', Sex='male', Age=36.0, SibSp=0, Parch=1, Ticket='PC 17755', Fare=512.3292, Cabin='B51 B53 B55', Embarked='C'), Row(PassengerId=738, Survived=1, Pclass=1, Name='Lesurer, Mr. Gustave J', Sex='male', Age=35.0, SibSp=0, Parch=0, Ticket='PC 17755', Fare=512.3292, Cabin='B101', Embarked='C')]
Thought: I now know the final answer
Final Answer: The passengers with the most expensive ticket 

'The passengers with the most expensive ticket were Miss. Anna Ward, Mr. Thomas Drake Martinez Cardeza, and Mr. Gustave J Lesurer.'